In [2]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import exp

np.set_printoptions(suppress=True)

# Gastric Cancer Data 

Adapted from [Codes for Unit 8: gastric.odc](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 8 Lesson 6](https://www.youtube.com/watch?v=t4pHpZxtC0U&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=87).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/gastric.csv).

Stablein et al. (1981) provide data on 90 patients affected by locally advanced, nonresectable gastric carcinoma. The patients are randomized to two treatments: chemotherapy alone (coded as 0) and chemotherapy plus radiation (coded as 1). Survival time is reported in days. Recorded times are censored if the patient stopped participating in the study before it finished.

Stablein, D. M., Carter, W. H., Novak, J. W. (1981). Analysis of survival data with nonproportional hazard functions. Control. Clin. Trials,  2 , 2, 149--159.

## Data
Columns are, from left to right:
- type: Treatment type, chemotherapy (0) or chemotherapy + radiation (1)
- censored: If censored, meaning the patient survived the observation period, the time in days appears here rather than in the times column. 0 if not censored.
- times: Recorded days without cancer recurrence. NaN if censored.

## Model changes
I didn't implement S, f, or h from the original model. They should be simple enough, but I really just wanted to get another example of censoring up before HW6. I will add those later.

PyMC really did not like the noninformative exponential prior on v (α in this model). To avoid the divide by zero errors, I just kept increasing lambda until the model ran all the way through. This is not ideal, but I haven't had time to look into it further. The results actually came out fairly close to the BUGS results.

## Method 1: ```pm.Censored```

The way PyMC censoring works is described in some detail in [this notebook](https://docs.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-truncated-censored-regression.html#censored-regression-model) by [Dr. Benjamin T. Vincent](https://github.com/drbenvincent). This is accomplished in the code [here](https://github.com/aesara-devs/aeppl/blob/751979802f1aef5478fdbf7cc1839df07df60825/aeppl/truncation.py#L79) if you want to take a look. For right-censoring, try this: ```pm.Censored("name", dist, lower=None, upper=censored, observed=y)```. The censored values can be an array of the same shape as the y values. 

If the y value equals the right-censored value, [```pm.Censored```](https://docs.pymc.io/en/latest/api/distributions/generated/pymc.Censored.html#pymc.Censored) returns the complement to the CDF evaluated at the censored value. If the y value is greater than the censored value, it returns ```-np.inf```. Otherwise, the distribution you passed to the ```dist``` parameter works as normal. What I've been doing is setting the values in the censored array to ```np.inf``` if the corresponding y value is not censored, and equal to the y value if it should be censored.



In [ ]:
data = np.loadtxt("./data/gastric.txt")
data.shape

FileNotFoundError: ./data/gastric.txt not found.

In [21]:
x = data[:, 0].copy()
censored = data[:, 1].copy()
y = data[:, 2].copy()
# for pymc, right-censored values must be greater than or equal to than the "upper" value
y[np.isnan(y)] = censored[np.isnan(y)]
censored[censored == 0] = np.inf

In [378]:
y

array([  17.,   42.,   44.,   48.,   60.,   72.,   74.,   95.,  103.,
        108.,  122.,  144.,  167.,  170.,  183.,  185.,  193.,  195.,
        197.,  208.,  234.,  235.,  254.,  307.,  315.,  401.,  445.,
        464.,  484.,  528.,  542.,  567.,  577.,  580.,  795.,  855.,
        882.,  892., 1031., 1033., 1306., 1335., 1366., 1452., 1472.,
          1.,   63.,  105.,  129.,  182.,  216.,  250.,  262.,  301.,
        301.,  342.,  354.,  356.,  358.,  380.,  381.,  383.,  383.,
        388.,  394.,  408.,  460.,  489.,  499.,  524.,  529.,  535.,
        562.,  675.,  676.,  748.,  748.,  778.,  786.,  797.,  945.,
        955.,  968., 1180., 1245., 1271., 1277., 1397., 1512., 1519.])

In [347]:
censored

array([  inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
        882.,  892., 1031., 1033., 1306., 1335.,   inf, 1452., 1472.,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,  381.,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,  529.,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,  945.,
         inf,   inf, 1180.,   inf,   inf, 1277., 1397., 1512., 1519.])

```{warning}
PyMC and BUGS do not specify the Weibull distribution in the same way!

α = v
β = λ ** (-1 / α)

```

In [ ]:
with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.0001)
    beta1 = pm.Normal("beta1", 0, tau=0.0001)
    α = pm.Exponential("α", 3)

    λ = exp(beta0 + beta1 * x)
    β = λ ** (-1 / α)

    obs_latent = pm.Weibull.dist(alpha=α, beta=β)
    likelihood = pm.Censored(
        "likelihood",
        obs_latent,
        lower=None,
        upper=censored,
        observed=y,
    )

    median0 = pm.Deterministic("median0", (np.log(2) * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (np.log(2) * exp(-beta0 - beta1)) ** (1 / α))

    trace = pm.sample(10000, tune=2000, cores=4, init="auto", step=[pm.Metropolis])

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>ABCMeta: []
>NUTS: [beta0, beta1, α]


In [397]:
az.summary(trace, hdi_prob=0.9)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-6.617,0.653,-7.693,-5.560,0.006,0.004,12089.0,14776.0,1.0
beta1,0.261,0.237,-0.120,0.652,0.002,0.001,16776.0,17473.0,1.0
α,1.001,0.096,0.845,1.158,0.001,0.001,12390.0,15220.0,1.0
median0,519.732,90.135,378.011,665.981,0.616,0.435,21255.0,23280.0,1.0
median1,400.156,71.182,287.236,515.158,0.476,0.338,22376.0,24715.0,1.0


## Method 2: Imputed Censoring

This method is from [this notebook](https://docs.pymc.io/projects/examples/en/latest/survival_analysis/censored_data.html#censored-data-model1) by [Luis Mario Domenzain](https://github.com/domenzain), [George Ho](https://github.com/eigenfoo), and [Dr. Ben Vincent](https://github.com/drbenvincent). This method imputes the values using what is almost another likelihood (not sure if it actually meets the definition of one, so I'm calling the variable ```impute_censored```) in the model, with the right-censored values as lower bounds. Since the two likelihoods share the same priors, this ends up working nearly as well as the previous method. Actually better, in the case of homework 6 question 2, since the previous method doesn't seem to work for the required model at all!

In [11]:
data = np.loadtxt("../data/gastric.txt")
x = data[:, 0].copy()
censored_vals = data[:, 1].copy()
y = data[:, 2].copy()

not_observed_mask = censored_vals == 0
censored = censored_vals[not_observed_mask]
y_uncensored = y[~not_observed_mask]
x_censored = x[not_observed_mask]
x_uncensored = x[~not_observed_mask]

In [12]:
with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.0001)
    beta1 = pm.Normal("beta1", 0, tau=0.0001)
    α = pm.Exponential("α", 1)

    λ_censored = exp(beta0 + beta1 * x_censored)
    β_censored = λ_censored ** (-1 / α)

    λ_uncensored = exp(beta0 + beta1 * x_uncensored)
    β_uncensored = λ_uncensored ** (-1 / α)

    impute_censored = pm.Bound(
        "impute_censored",
        pm.Weibull.dist(alpha=α, beta=β_censored),
        lower=censored,
        shape=censored.shape[0],
    )

    likelihood = pm.Weibull(
        "likelihood",
        alpha=α,
        beta=β_uncensored,
        observed=y_uncensored,
        shape=y_uncensored.shape[0],
    )

    median0 = pm.Deterministic("median0", (np.log(2) * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (np.log(2) * exp(-beta0 - beta1)) ** (1 / α))

    trace = pm.sample(
        10000, tune=2000, cores=4, init="auto", step=[pm.NUTS(target_accept=0.9)]
    )

NameError: name 'β_cens' is not defined

In [19]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-6.747,0.662,-8.059,-5.470,0.004,0.003,22386.0,22192.0,1.0
beta1,0.263,0.233,-0.194,0.723,0.001,0.001,42748.0,29654.0,1.0
α,1.021,0.097,0.834,1.212,0.001,0.000,22191.0,21764.0,1.0
right_censored[0],1451.379,605.188,882.017,2630.015,2.814,2.098,29824.0,16865.0,1.0
right_censored[1],1467.201,609.670,892.031,2661.425,2.886,2.138,30214.0,16918.0,1.0
right_censored[2],1599.929,609.118,1031.034,2757.275,2.816,2.095,30065.0,18023.0,1.0
right_censored[3],1598.762,602.081,1033.027,2750.599,2.789,2.064,32297.0,18540.0,1.0
right_censored[4],1874.160,609.180,1306.018,3037.794,2.823,2.087,30803.0,19025.0,1.0
right_censored[5],1902.432,605.028,1335.010,3053.389,2.852,2.104,32254.0,18788.0,1.0
right_censored[6],2022.598,624.856,1452.038,3220.435,2.955,2.161,26631.0,14986.0,1.0


In [16]:
r_censored

array([ 882.,  892., 1031., 1033., 1306., 1335., 1452., 1472.,  381.,
        529.,  945., 1180., 1277., 1397., 1512., 1519.])